In [ ]:
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, CallbackContext
from PyPDF2 import PdfReader, PdfWriter
import os

def lock_pdf(input_path, output_path, password):
    try:
        reader = PdfReader(input_path)
        writer = PdfWriter()

        # Copy all pages to the writer
        for page in reader.pages:
            writer.add_page(page)

        # Set the password
        writer.encrypt(password)

        # Write the locked PDF
        with open(output_path, "wb") as output_file:
            writer.write(output_file)
        return True, None
    except Exception as e:
        return False, str(e)

async def start(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text("Hello! Send me a PDF file, and I'll lock it with a password.")
async def handle_pdf(update: Update, context: CallbackContext) -> None:
    # Check if the received file is a document
    if update.message.document:
        document = update.message.document
        if document.mime_type == "application/pdf":
            file_id = document.file_id
            file_name = document.file_name

            # Define the OneDrive directory for temporary storage
            onedrive_temp_dir = "C:/Users/Locas/OneDrive/ShareTempPDF"
            os.makedirs(onedrive_temp_dir, exist_ok=True)

            # Define file paths for input and output in OneDrive
            input_path = os.path.join(onedrive_temp_dir, file_name)
            output_path = os.path.join(onedrive_temp_dir, f"locked_{file_name}")

            # Download the file to OneDrive
            file = await context.bot.get_file(file_id)
            await file.download_to_drive(input_path)

            # Store paths in user_data for later use
            context.user_data["input_path"] = input_path
            context.user_data["output_path"] = output_path

            # Notify the user
            await update.message.reply_text("Send me a password to lock your PDF.")
        else:
            await update.message.reply_text("Please send a valid PDF file.")

async def handle_password(update: Update, context: CallbackContext) -> None:
    password = update.message.text

    # Ensure input_path and output_path exist in user data
    if "input_path" in context.user_data and "output_path" in context.user_data:
        input_path = context.user_data["input_path"]
        output_path = context.user_data["output_path"]

        # Lock the PDF
        success, error = lock_pdf(input_path, output_path, password)

        if success:
            # Send the locked PDF back
            await update.message.reply_document(document=open(output_path, "rb"), filename=os.path.basename(output_path))
        else:
            await update.message.reply_text(f"Error locking PDF: {error}")

        # Clean up temporary files
        os.remove(input_path)
        os.remove(output_path)
        context.user_data.clear()
    else:
        await update.message.reply_text("Please send a PDF first.")
async def main():
    application = ApplicationBuilder().token("YOUR TOKEN").build()

    # Add command and message handlers
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.Document.PDF, handle_pdf))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_password))

    # Run the bot

await application.run_polling()
import nest_asyncio
nest_asyncio.apply()
await main()


